In [2]:
!pip install pydantic


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.23.4-cp312-none-win_amd64.whl.metadata (6.7 kB)
  Using cached typing_extensions-4.12.2-py3-none-any.whl.metadata (3.0 kB)
Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
Using cached pydantic_core-2.23.4-cp312-none-win_amd64.whl (1.9 MB)
Using cached annotated_types-0.7.0-py3-none-any.whl (13 kB)
Using cached typing_extensions-4.12.2-py3-none-any.whl (37 kB)


In [3]:
import pydantic
print(pydantic.__version__)

2.9.2


Validation without Pydantic

In [4]:
class User:
    def __init__(self, id, name='Jane Doe'):
        if not isinstance(id, int):
            raise TypeError(f'Expected id to be an int, got {type(id).__name__}')
        
        if not isinstance(name, str):
            raise TypeError(f'Expected name to be a str, got {type(name).__name__}')
        
        self.id = id
        self.name = name

try:
    user = User(id='123')
except TypeError as e:
    print(e)


Expected id to be an int, got str


Validation with Pydantic

In [6]:
from pydantic import BaseModel

class User(BaseModel):
    id: int
    name: str = 'Jane Doe'


In [7]:
user = User(id='123')

In [8]:
print(user.id)

123


In [9]:
print(user.model_fields_set)
user = User(id='123', name='Joe Doe')
print(user.model_fields_set)


{'id'}
{'id', 'name'}


In [13]:
# Generate Dictionary
print(user.model_dump())

# Generate JSON
print(user.model_dump_json())

# Generate JSONSchema
print(user.model_json_schema())

{'id': 123, 'name': 'Joe Doe'}
{"id":123,"name":"Joe Doe"}
{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'default': 'Jane Doe', 'title': 'Name', 'type': 'string'}}, 'required': ['id'], 'title': 'User', 'type': 'object'}


### Nested models

In [14]:
from typing import List, Optional
from pydantic import BaseModel


class Food(BaseModel):
    name: str
    price: float
    ingredients: Optional[List[str]] = None


class Restaurant(BaseModel):
    name: str
    location: str
    foods: List[Food]


restaurant_instance = Restaurant(
    name="Tasty Bites",
    location="123, Flavor Street",
    foods=[
        {"name": "Cheese Pizza", "price": 12.50, "ingredients": ["Cheese", "Tomato Sauce", "Dough"]},
        {"name": "Veggie Burger", "price": 8.99}
    ]
)

print(restaurant_instance)
print(restaurant_instance.model_dump())

name='Tasty Bites' location='123, Flavor Street' foods=[Food(name='Cheese Pizza', price=12.5, ingredients=['Cheese', 'Tomato Sauce', 'Dough']), Food(name='Veggie Burger', price=8.99, ingredients=None)]
{'name': 'Tasty Bites', 'location': '123, Flavor Street', 'foods': [{'name': 'Cheese Pizza', 'price': 12.5, 'ingredients': ['Cheese', 'Tomato Sauce', 'Dough']}, {'name': 'Veggie Burger', 'price': 8.99, 'ingredients': None}]}


In [18]:
# Alternative

f1 = Food(name= "Cheese Pizza", price= 12.50)
f2 = Food(name= "Veggie Burger", price= 10.50)
restaurant_instance2 =  Restaurant(
    name="Tasty Bites",
    location="123, Flavor Street",
    foods=[f1,f2])

In [20]:
restaurant_instance2.model_dump()

{'name': 'Tasty Bites',
 'location': '123, Flavor Street',
 'foods': [{'name': 'Cheese Pizza', 'price': 12.5, 'ingredients': None},
  {'name': 'Veggie Burger', 'price': 10.5, 'ingredients': None}]}

In [15]:
!pip install pydantic[email]


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
from typing import List
from pydantic import BaseModel, EmailStr, PositiveInt, conlist, Field, HttpUrl

class Address(BaseModel):
    street: str
    city: str
    state: str
    zip_code: str

class Employee(BaseModel):
    name: str
    position: str
    email: EmailStr
    
class Owner(BaseModel):
    name: str
    email: EmailStr
    
class Restaurant(BaseModel):
    name: str = Field(..., pattern=r"^[a-zA-Z0-9-' ]+$")
    owner: Owner
    address: Address
    employees: conlist(Employee, min_length=2)
    number_of_seats: PositiveInt
    delivery: bool
    website: HttpUrl

# Creating an instance of the Restaurant class
restaurant_instance = Restaurant(
    name="Tasty Bites",
    owner={
        "name": "John Doe",
        "email": "john.doe@example.com"
    },
    address={
        "street": "123, Flavor Street",
        "city": "Tastytown",
        "state": "TS",
        "zip_code": "12345",
    },
    employees=[
        {
            "name": "Jane Doe",
            "position": "Chef",
            "email": "jane.doe@example.com"
        },
        {
            "name": "Mike Roe",
            "position": "Waiter",
            "email": "mike.roe@example.com"
        }
    ],
    number_of_seats=50,
    delivery=True,
    website="http://tastybites.com"
)

# Printing the instance
print(restaurant_instance)


name='Tasty Bites' owner=Owner(name='John Doe', email='john.doe@example.com') address=Address(street='123, Flavor Street', city='Tastytown', state='TS', zip_code='12345') employees=[Employee(name='Jane Doe', position='Chef', email='jane.doe@example.com'), Employee(name='Mike Roe', position='Waiter', email='mike.roe@example.com')] number_of_seats=50 delivery=True website=Url('http://tastybites.com/')


Field Validators

In [24]:
from pydantic import BaseModel, EmailStr, field_validator

class Owner(BaseModel):
    name: str
    email: EmailStr
    
    @field_validator('name')
    @classmethod
    def name_must_contain_space(cls, v: str) -> str:
        if ' ' not in v:
            raise ValueError('Owner name must contain a space')
        return v.title()

try:
    owner_instance = Owner(name="JohnDoe", email="john.doe@example.com")
except ValueError as e:
    print(e)



1 validation error for Owner
name
  Value error, Owner name must contain a space [type=value_error, input_value='JohnDoe', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


Model validators - allowing you to create a model before and after field validation

In [25]:
from typing import Any
from pydantic import BaseModel, EmailStr, ValidationError, model_validator

class Owner(BaseModel):
    name: str
    email: EmailStr
    
    @model_validator(mode='before')
    @classmethod
    def check_sensitive_info_omitted(cls, data: Any) -> Any:
        if isinstance(data, dict):
            if 'password' in data:
                raise ValueError('password should not be included')
            if 'card_number' in data:
                raise ValueError('card_number should not be included')
        return data
    
    @model_validator(mode='after')
    def check_name_contains_space(self) -> 'Owner':
        if ' ' not in self.name:
            raise ValueError('Owner name must contain a space')
        return self



print(Owner(name="John Doe", email="john.doe@example.com")) 

try:
    Owner(name="JohnDoe", email="john.doe@example.com", password="password123")
except ValidationError as e:
    print(e) 


name='John Doe' email='john.doe@example.com'
1 validation error for Owner
  Value error, password should not be included [type=value_error, input_value={'name': 'JohnDoe', 'emai...assword': 'password123'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


In [26]:
try:
    Owner(name="JohnDoe", email="john.doe@example.com")
except ValidationError as e:
    print(e)

1 validation error for Owner
  Value error, Owner name must contain a space [type=value_error, input_value={'name': 'JohnDoe', 'emai... 'john.doe@example.com'}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.9/v/value_error


Fields - Used to customize and add metadata to fields of models

In [27]:
from pydantic import BaseModel, Field

class User(BaseModel):
    name: str = Field(default='John Doe')

user = User()
print(user)

name='John Doe'


In [28]:
from uuid import uuid4

from pydantic import BaseModel, Field

class User(BaseModel):
    id: int = Field(default_factory=lambda: uuid4().hex)

user = User()
print(user)

id='d18a09cf42a74300a09773eea395b239'


Field aliases - For validation and serialization, you can define an alias for a field.

There are three ways to define an alias:

- `Field(..., alias='foo')`
- `Field(..., validation_alias='foo')`
- `Field(..., serialization_alias='foo')`

In [30]:
from pydantic import BaseModel, Field


class User(BaseModel):
    name: str = Field(..., alias='username')


user = User(username='johndoe')  
print(user)
print(user.model_dump(by_alias=True))
print(user.model_dump(by_alias=False))

name='johndoe'
{'username': 'johndoe'}
{'name': 'johndoe'}


But why would you want to do it? For example if you API and database fields differ - you only need one model

Lets look at field constraints

In [33]:
from typing import List
from pydantic import BaseModel, Field, EmailStr
from decimal import Decimal

class User(BaseModel):
    username: str = Field(..., min_length=3, max_length=10, pattern=r'^\w+$')
    email: EmailStr = Field(...)
    age: int = Field(..., gt=0, le=120)
    height: float = Field(..., gt=0.0)
    is_active: bool = Field(True)
    balance: Decimal = Field(..., max_digits=10, decimal_places=2)
    favorite_numbers: List[int] = Field(..., min_items=1)


In [34]:
user_instance = User(
    username="john_doe",
    age=30,
    height=5.9,
    weight=160.5,
    email="john.doe@example.com",
    password="securepassword",
    balance=9999.99,
    favorite_numbers=[1,2,3]
)

print(user_instance)

username='john_doe' email='john.doe@example.com' age=30 height=5.9 is_active=True balance=Decimal('9999.99') favorite_numbers=[1, 2, 3]


Computed Fields

In [35]:
from pydantic import BaseModel, computed_field
from datetime import datetime


class Person(BaseModel):
    name: str
    birth_year: int

    @computed_field
    @property
    def age(self) -> int:
        current_year = datetime.now().year
        return current_year - self.birth_year


print(Person(name="John Doe", birth_year=2000).model_dump())


{'name': 'John Doe', 'birth_year': 2000, 'age': 24}


In [36]:
from pydantic import BaseModel, ValidationError, field_validator
from datetime import datetime

class Person(BaseModel):
    name: str
    birth_year: int

    @property
    def age(self) -> int:
        current_year = datetime.now().year
        return current_year - self.birth_year

    @field_validator('birth_year')
    @classmethod
    def validate_age(cls, v: int) -> int:
        current_year = datetime.now().year
        if current_year - v < 18:
            raise ValueError('Person must be 18 years or older')
        return v

try:
    print(Person(name="John Doe", birth_year=2006))
except ValidationError as e:
    print(e)


name='John Doe' birth_year=2006


You also also use dataclasses and pydantics valiation logic - dataclasses do not provide that out of the box

In [37]:
from dataclasses import dataclass, field
from typing import List, Optional
from pydantic import Field, TypeAdapter


@dataclass
class User:
    id: int
    name: str = 'John Doe'
    age: Optional[int] = field(
        default=None,
        metadata=dict(title='The age of the user', description='do not lie!', ge=18),
    )
    height: Optional[int] = Field(None, title='The height in cm', ge=50, le=300)
    friends: List[int] = field(default_factory=lambda: [0])





TypeAdapter

You may have types that are not `BaseModels` that you want to validate data against. Or you may want to validate a `List[SomeModel]`, or dump it to JSON.

For use cases like this, Pydantic provides `TypeAdapter`, which can be used for type validation, serialization, and JSON schema generation without creating a `BaseModel`.


In [38]:
# Example of using TypeAdapter to get json_schema of the User dataclass
print(TypeAdapter(User).json_schema())

{'properties': {'id': {'title': 'Id', 'type': 'integer'}, 'name': {'default': 'John Doe', 'title': 'Name', 'type': 'string'}, 'age': {'anyOf': [{'minimum': 18, 'type': 'integer'}, {'type': 'null'}], 'default': None, 'description': 'do not lie!', 'title': 'The age of the user'}, 'height': {'anyOf': [{'maximum': 300, 'minimum': 50, 'type': 'integer'}, {'type': 'null'}], 'default': None, 'title': 'The height in cm'}, 'friends': {'items': {'type': 'integer'}, 'title': 'Friends', 'type': 'array'}}, 'required': ['id'], 'title': 'User', 'type': 'object'}


### Strict mode

Pydantic, by default, tries to coerce values to the declared type, converting inputs like the string "123" to integer 123, but this automatic conversion can be undesirable in situations where strict type compliance is required, causing a need for configurations to make Pydantic error out instead.

In [39]:
from pydantic import BaseModel, ValidationError


class User(BaseModel):
    id: int
    username: str


print(User.model_validate({'id': '42', 'username': 'john_doe'}))  # lax mode
#> id=42 username='john_doe'

id=42 username='john_doe'


In [40]:
try:
    User.model_validate({'id': '42', 'username': 'john_doe'}, strict=True)  # strict mode
except ValidationError as exc:
    print(exc)

1 validation error for User
id
  Input should be a valid integer [type=int_type, input_value='42', input_type=str]
    For further information visit https://errors.pydantic.dev/2.9/v/int_type


Settings Management 

Pydantic Settings provides optional Pydantic features for loading a settings or config class from environment variables or secrets files.

In [41]:
!pip install pydantic-settings

  Using cached python_dotenv-1.0.1-py3-none-any.whl.metadata (23 kB)
Using cached python_dotenv-1.0.1-py3-none-any.whl (19 kB)



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
from dotenv import load_dotenv
load_dotenv()

True

In [59]:
import os
from pydantic import Field
from pydantic_settings import BaseSettings


class Settings(BaseSettings):
    auth_key: str = Field(validation_alias='my_auth_key')  
    api_key: str = Field(alias='my_api_key')  
    

print(Settings().model_dump())

{'auth_key': 'env_auth_key', 'api_key': 'test'}


In [60]:
import os
from pydantic import Field, AliasChoices
from pydantic_settings import BaseSettings

os.environ["AUTH_KEY"] = "test_auth_key"
os.environ["MY_API_KEY"] = "test"
os.environ["ENV2"] = "https://mysuperurl.com"


class Settings(BaseSettings):
    service_name: str = Field(default="default")
    auth_key: str  
    api_key: str = Field(alias='my_api_key')  
    url: str = Field(validation_alias=AliasChoices("env1", "env2"))
     

print(Settings().model_dump())

{'service_name': 'default', 'auth_key': 'test_auth_key', 'api_key': 'test', 'url': 'https://mysuperurl.com'}


In [68]:
import os
from pydantic import Field
from pydantic_settings import BaseSettings, SettingsConfigDict

# Set environment variables with the prefix
os.environ["PRODUCTION_AUTH_KEY"] = "test_auth_key"
os.environ["PRODUCTION_MY_API_KEY"] = "test"
os.environ["PRODUCTION_ENV2"] = "https://mysuperurl.com"


class Settings(BaseSettings):
    model_config = SettingsConfigDict(env_prefix='production_')

    service_name: str = Field(default="default")
    auth_key: str
    api_key: str = Field(alias='my_api_key')
    url: str = Field(validation_alias=AliasChoices("env1", "env2"))

print(Settings().model_dump())


{'service_name': 'default', 'auth_key': 'test_auth_key', 'api_key': 'test', 'url': 'https://mysuperurl.com'}


You can also use a .env file

In [66]:
from pydantic_settings import BaseSettings, SettingsConfigDict


class Settings(BaseSettings):

    model_config = SettingsConfigDict(env_file='.env', env_file_encoding='utf-8')

    service_name: str = Field(default="default")
    auth_key: str  = Field(alias='my_auth_key')  
    api_key: str = Field(alias='my_api_key')  

print(Settings().model_dump())

{'service_name': 'default', 'auth_key': 'env_auth_key', 'api_key': 'test'}


Extra attributes (https://docs.pydantic.dev/2.3/usage/model_config/#extra-attributes)

In [67]:
from pydantic_settings import BaseSettings, SettingsConfigDict


class Settings(BaseSettings):

    model_config = SettingsConfigDict(env_file='.env', env_file_encoding='utf-8', extra="ignore") # forbid, allow

    service_name: str = Field(default="default")
    auth_key: str  
    api_key: str = Field(alias='my_api_key')  
     

print(Settings().model_dump())

{'service_name': 'default', 'auth_key': 'test_auth_key', 'api_key': 'test'}
